# Evolver Loop 3 Analysis

## Critical Issue: Both submissions failed with 'Overlapping trees in group 008'

We need to:
1. Understand the EXACT overlap detection the LB uses
2. Investigate group 008 specifically
3. Find a truly valid submission

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')

print('Libraries loaded')

Libraries loaded


In [2]:
# Tree geometry constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

BASE_TREE_VERTICES = np.array([
    [0.0, TIP_Y],
    [TOP_W/2, TIER_1_Y],
    [TOP_W/4, TIER_1_Y],
    [MID_W/2, TIER_2_Y],
    [MID_W/4, TIER_2_Y],
    [BASE_W/2, BASE_Y],
    [TRUNK_W/2, BASE_Y],
    [TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, BASE_Y],
    [-BASE_W/2, BASE_Y],
    [-MID_W/4, TIER_2_Y],
    [-MID_W/2, TIER_2_Y],
    [-TOP_W/4, TIER_1_Y],
    [-TOP_W/2, TIER_1_Y],
])

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])
    rotated = BASE_TREE_VERTICES @ rotation_matrix.T
    translated = rotated + np.array([x, y])
    return Polygon(translated)

def parse_submission(df):
    result = df.copy()
    for col in ['x', 'y', 'deg']:
        result[col] = result[col].str.replace('s', '').astype(float)
    return result

print('Helper functions defined')

Helper functions defined


In [3]:
# LB-style overlap check: intersects() and not touches()
def check_lb_overlaps(polygons):
    """Check for overlaps using LB's method: intersects() and not touches()"""
    overlaps = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                overlaps.append((i, j))
    return overlaps

# Also check with area-based method for comparison
def check_area_overlaps(polygons):
    """Check for overlaps using area > 0 method"""
    overlaps = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            intersection = polygons[i].intersection(polygons[j])
            if intersection.area > 0:
                overlaps.append((i, j, intersection.area))
    return overlaps

print('Overlap check functions defined')

Overlap check functions defined


In [4]:
# Load the current submission and check group 008 specifically
print('Loading current submission...')
current_df = pd.read_csv('/home/submission/submission.csv')
current_parsed = parse_submission(current_df)
print(f'Shape: {current_parsed.shape}')

# Check group 008
prefix = '008_'
config_df = current_parsed[current_parsed['id'].str.startswith(prefix)]
print(f'\nGroup 008 has {len(config_df)} trees')
print(config_df)

Loading current submission...
Shape: (20100, 4)

Group 008 has 8 trees
       id         x         y         deg
28  008_0 -0.249562 -0.411092   51.766738
29  008_1  0.664668 -0.857305  113.629378
30  008_2 -0.664668  0.257305  293.629378
31  008_3  0.557305  0.326837  203.629378
32  008_4  0.249562 -0.188908  231.766738
33  008_5 -0.206589 -0.855262  293.629378
34  008_6  0.206589  0.255262  113.629378
35  008_7 -0.557305 -0.926837   23.629378


In [5]:
# Create polygons for group 008 and check overlaps
polygons_008 = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]

lb_overlaps = check_lb_overlaps(polygons_008)
area_overlaps = check_area_overlaps(polygons_008)

print(f'LB-style overlaps in group 008: {len(lb_overlaps)}')
if lb_overlaps:
    print(f'Overlapping pairs: {lb_overlaps}')

print(f'\nArea-based overlaps in group 008: {len(area_overlaps)}')
if area_overlaps:
    for i, j, area in area_overlaps:
        print(f'  Trees {i} and {j}: area = {area}')

LB-style overlaps in group 008: 0

Area-based overlaps in group 008: 0


In [6]:
# Check ALL groups with LB-style overlap detection
print('Checking ALL groups with LB-style overlap detection...')

lb_overlap_groups = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = current_parsed[current_parsed['id'].str.startswith(prefix)]
    if len(config_df) != n:
        print(f'WARNING: Group {n} has {len(config_df)} trees instead of {n}')
        continue
    
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    overlaps = check_lb_overlaps(polygons)
    if overlaps:
        lb_overlap_groups.append((n, len(overlaps)))

print(f'\nGroups with LB-style overlaps: {len(lb_overlap_groups)}')
if lb_overlap_groups:
    print('First 20 groups with overlaps:')
    for n, count in lb_overlap_groups[:20]:
        print(f'  Group {n:03d}: {count} overlapping pairs')

Checking ALL groups with LB-style overlap detection...



Groups with LB-style overlaps: 0


In [7]:
# Check chistyakov_best.csv - the evaluator says this has 0 overlaps
print('\nChecking chistyakov_best.csv...')
chistyakov_df = pd.read_csv('/home/code/preoptimized/chistyakov_best.csv')
chistyakov_parsed = parse_submission(chistyakov_df)

# Check group 008 in chistyakov
prefix = '008_'
config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith(prefix)]
polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
overlaps = check_lb_overlaps(polygons)
print(f'Group 008 in chistyakov_best.csv: {len(overlaps)} LB-style overlaps')

# Check ALL groups
chistyakov_overlap_groups = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith(prefix)]
    if len(config_df) != n:
        continue
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    overlaps = check_lb_overlaps(polygons)
    if overlaps:
        chistyakov_overlap_groups.append((n, len(overlaps)))

print(f'Total groups with LB-style overlaps in chistyakov_best.csv: {len(chistyakov_overlap_groups)}')
if chistyakov_overlap_groups:
    print(f'Groups: {chistyakov_overlap_groups}')


Checking chistyakov_best.csv...
Group 008 in chistyakov_best.csv: 0 LB-style overlaps


Total groups with LB-style overlaps in chistyakov_best.csv: 0


In [8]:
# The LB is rejecting with "Overlapping trees in group 008" but our check shows 0 overlaps
# This could mean:
# 1. The LB uses different tree geometry
# 2. The LB uses different precision
# 3. There's a floating point issue in how we read the CSV

# Let's check the raw CSV values for group 008
print('Raw CSV values for group 008:')
raw_df = pd.read_csv('/home/submission/submission.csv')
group_008 = raw_df[raw_df['id'].str.startswith('008_')]
print(group_008)

# Check if there are any precision issues
print('\nParsed values:')
for _, row in group_008.iterrows():
    x = float(row['x'].replace('s', ''))
    y = float(row['y'].replace('s', ''))
    deg = float(row['deg'].replace('s', ''))
    print(f"{row['id']}: x={x:.20f}, y={y:.20f}, deg={deg:.20f}")

Raw CSV values for group 008:
       id                      x                     y                  deg
28  008_0  s-0.24956221975123125  s-0.4110915693917769    s51.7667378176506
29  008_1    s0.6646675428904323  s-0.8573053542961684  s113.62937778579037
30  008_2   s-0.6646675428602861   s0.2573053553705201   s293.6293777306379
31  008_3     s0.557305355195559   s0.3268368934018002  s203.62937773065627
32  008_4    s0.2495622198407891  s-0.1889084299202516   s231.7667378027784
33  008_5   s-0.2065888753430593  s-0.8552621014916894   s293.6293777857903
34  008_6   s0.20658887514113536   s0.2552621014870126  s113.62937773063783
35  008_7   s-0.5573053551764364  s-0.9268368933098561  s23.629377730657012

Parsed values:
008_0: x=-0.24956221975123124568, y=-0.41109156939177687873, deg=51.76673781765060056159
008_1: x=0.66466754289043228354, y=-0.85730535429616838705, deg=113.62937778579036773863
008_2: x=-0.66466754286028606469, y=0.25730535537052007244, deg=293.62937773063788426953
008

In [9]:
# Let's check what the LB might be using for tree geometry
# Maybe the LB uses a slightly different tree definition

# Let's also check if there's a buffer/tolerance issue
# Try checking with a small buffer around polygons

print('Checking with buffered polygons (simulating potential LB tolerance)...')

for buffer_size in [1e-10, 1e-9, 1e-8, 1e-7, 1e-6]:
    config_df = current_parsed[current_parsed['id'].str.startswith('008_')]
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']).buffer(buffer_size) for _, row in config_df.iterrows()]
    overlaps = check_lb_overlaps(polygons)
    print(f'Buffer {buffer_size}: {len(overlaps)} overlaps')
    if overlaps:
        print(f'  Pairs: {overlaps}')

Checking with buffered polygons (simulating potential LB tolerance)...
Buffer 1e-10: 10 overlaps
  Pairs: [(0, 5), (0, 6), (0, 7), (1, 5), (2, 6), (3, 4), (3, 6), (4, 5), (4, 6), (5, 7)]
Buffer 1e-09: 10 overlaps
  Pairs: [(0, 5), (0, 6), (0, 7), (1, 5), (2, 6), (3, 4), (3, 6), (4, 5), (4, 6), (5, 7)]
Buffer 1e-08: 10 overlaps
  Pairs: [(0, 5), (0, 6), (0, 7), (1, 5), (2, 6), (3, 4), (3, 6), (4, 5), (4, 6), (5, 7)]
Buffer 1e-07: 10 overlaps
  Pairs: [(0, 5), (0, 6), (0, 7), (1, 5), (2, 6), (3, 4), (3, 6), (4, 5), (4, 6), (5, 7)]
Buffer 1e-06: 10 overlaps
  Pairs: [(0, 5), (0, 6), (0, 7), (1, 5), (2, 6), (3, 4), (3, 6), (4, 5), (4, 6), (5, 7)]


In [10]:
# The trees are exactly touching! The LB might use slightly different precision
# Let's check the actual distance between polygons

print('Checking actual distances between polygons in group 008...')
config_df = current_parsed[current_parsed['id'].str.startswith('008_')]
polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]

for i in range(len(polygons)):
    for j in range(i+1, len(polygons)):
        dist = polygons[i].distance(polygons[j])
        intersects = polygons[i].intersects(polygons[j])
        touches = polygons[i].touches(polygons[j])
        if dist < 1e-6:  # Very close
            print(f'Trees {i} and {j}: distance={dist:.2e}, intersects={intersects}, touches={touches}')

Checking actual distances between polygons in group 008...
Trees 0 and 5: distance=1.65e-15, intersects=False, touches=False
Trees 0 and 6: distance=6.06e-16, intersects=False, touches=False
Trees 0 and 7: distance=1.87e-10, intersects=False, touches=False
Trees 1 and 5: distance=1.22e-16, intersects=False, touches=False
Trees 2 and 6: distance=3.53e-16, intersects=False, touches=False
Trees 3 and 4: distance=4.01e-15, intersects=False, touches=False
Trees 3 and 6: distance=1.19e-14, intersects=False, touches=False
Trees 4 and 5: distance=9.23e-16, intersects=False, touches=False
Trees 4 and 6: distance=1.56e-15, intersects=False, touches=False
Trees 5 and 7: distance=1.08e-15, intersects=False, touches=False


In [11]:
# Check chistyakov_best.csv group 008 distances
print('Checking distances in chistyakov_best.csv group 008...')
config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith('008_')]
polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]

close_pairs = 0
for i in range(len(polygons)):
    for j in range(i+1, len(polygons)):
        dist = polygons[i].distance(polygons[j])
        if dist < 1e-6:  # Very close
            close_pairs += 1
            print(f'Trees {i} and {j}: distance={dist:.2e}')

print(f'\nTotal close pairs (< 1e-6): {close_pairs}')

Checking distances in chistyakov_best.csv group 008...
Trees 0 and 1: distance=3.86e-09
Trees 0 and 3: distance=4.33e-07
Trees 1 and 3: distance=2.06e-07
Trees 1 and 5: distance=7.76e-08
Trees 1 and 6: distance=4.59e-08
Trees 2 and 6: distance=1.17e-08
Trees 3 and 4: distance=4.13e-08
Trees 4 and 7: distance=3.67e-07

Total close pairs (< 1e-6): 8


In [12]:
# Check chistyakov_best.csv with buffer
print('Checking chistyakov_best.csv group 008 with buffer...')
config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith('008_')]

for buffer_size in [1e-10, 1e-9, 1e-8, 1e-7, 1e-6]:
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']).buffer(buffer_size) for _, row in config_df.iterrows()]
    overlaps = check_lb_overlaps(polygons)
    print(f'Buffer {buffer_size}: {len(overlaps)} overlaps')

Checking chistyakov_best.csv group 008 with buffer...
Buffer 1e-10: 0 overlaps
Buffer 1e-09: 0 overlaps
Buffer 1e-08: 2 overlaps
Buffer 1e-07: 5 overlaps
Buffer 1e-06: 8 overlaps


In [13]:
# Let's check ALL groups in chistyakov_best.csv with a 1e-10 buffer
# to see if it would pass the LB

print('Checking ALL groups in chistyakov_best.csv with 1e-10 buffer...')
buffer_size = 1e-10
overlap_groups = []

for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith(prefix)]
    if len(config_df) != n:
        continue
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']).buffer(buffer_size) for _, row in config_df.iterrows()]
    overlaps = check_lb_overlaps(polygons)
    if overlaps:
        overlap_groups.append((n, len(overlaps)))

print(f'Groups with overlaps (buffer 1e-10): {len(overlap_groups)}')
if overlap_groups:
    print(f'Groups: {overlap_groups[:20]}')

Checking ALL groups in chistyakov_best.csv with 1e-10 buffer...


Groups with overlaps (buffer 1e-10): 188
Groups: [(2, 1), (3, 1), (4, 1), (5, 3), (6, 5), (9, 1), (10, 2), (11, 2), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 2), (18, 1), (19, 2), (20, 2), (21, 1), (22, 2), (23, 1)]


In [14]:
# Find the minimum distance across ALL groups in chistyakov_best.csv
print('Finding minimum distances in chistyakov_best.csv...')

min_distances = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith(prefix)]
    if len(config_df) != n:
        continue
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            dist = polygons[i].distance(polygons[j])
            if dist < 1e-6:
                min_distances.append((n, i, j, dist))

print(f'Found {len(min_distances)} pairs with distance < 1e-6')
if min_distances:
    min_distances.sort(key=lambda x: x[3])
    print('Smallest distances:')
    for n, i, j, dist in min_distances[:20]:
        print(f'  Group {n:03d}, trees {i}-{j}: {dist:.2e}')

Finding minimum distances in chistyakov_best.csv...


Found 22539 pairs with distance < 1e-6
Smallest distances:
  Group 002, trees 0-1: 1.09e-16
  Group 003, trees 0-1: 6.82e-16
  Group 156, trees 66-101: 2.51e-15
  Group 185, trees 35-171: 4.12e-15
  Group 195, trees 38-135: 4.55e-15
  Group 028, trees 18-27: 4.71e-15
  Group 068, trees 2-59: 4.82e-15
  Group 082, trees 48-77: 4.88e-15
  Group 072, trees 13-62: 5.99e-15
  Group 176, trees 27-94: 6.81e-15
  Group 138, trees 94-112: 7.66e-15
  Group 130, trees 8-18: 8.72e-15
  Group 164, trees 93-109: 9.37e-15
  Group 169, trees 30-161: 9.70e-15
  Group 085, trees 41-48: 1.11e-14
  Group 037, trees 6-8: 1.17e-14
  Group 069, trees 14-30: 1.22e-14
  Group 006, trees 0-1: 1.47e-14
  Group 005, trees 0-1: 1.47e-14
  Group 032, trees 11-20: 1.77e-14


In [15]:
# The LB uses scale_factor = 1e15 for coordinates
# This means a distance of 1e-15 becomes 1 in scaled coordinates
# Let's check what the minimum safe distance should be

print('Understanding the precision issue:')
print('LB scale_factor = 1e15')
print('A distance of 1e-15 in real coords = 1 in scaled coords')
print('A distance of 1e-16 in real coords = 0.1 in scaled coords')
print()
print('Our minimum distances:')
print('  Current submission: 1e-15 to 1e-16 (essentially 0 in scaled coords)')
print('  chistyakov_best.csv: 1e-9 to 1e-7 (1e6 to 1e8 in scaled coords)')
print()
print('The LB likely considers trees overlapping if they are within 1 scaled unit')
print('This means we need minimum distance > 1e-15 in real coords')
print()
print('SOLUTION: Use chistyakov_best.csv which has better separation')

Understanding the precision issue:
LB scale_factor = 1e15
A distance of 1e-15 in real coords = 1 in scaled coords
A distance of 1e-16 in real coords = 0.1 in scaled coords

Our minimum distances:
  Current submission: 1e-15 to 1e-16 (essentially 0 in scaled coords)
  chistyakov_best.csv: 1e-9 to 1e-7 (1e6 to 1e8 in scaled coords)

The LB likely considers trees overlapping if they are within 1 scaled unit
This means we need minimum distance > 1e-15 in real coords

SOLUTION: Use chistyakov_best.csv which has better separation


In [16]:
# Let's copy chistyakov_best.csv to submission and test it
import shutil

# Copy chistyakov_best.csv to submission
shutil.copy('/home/code/preoptimized/chistyakov_best.csv', '/home/submission/submission.csv')
print('Copied chistyakov_best.csv to /home/submission/submission.csv')

# Verify the score
df = pd.read_csv('/home/submission/submission.csv')
parsed = parse_submission(df)

def get_bounding_box_side(polygons):
    if not polygons:
        return 0.0
    union = unary_union(polygons)
    bounds = union.bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    return max(width, height)

total_score = 0.0
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = parsed[parsed['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    side = get_bounding_box_side(polygons)
    total_score += (side ** 2) / n

print(f'Score: {total_score:.6f}')
print(f'Target: 68.896973')
print(f'Gap: {total_score - 68.896973:.6f}')

Copied chistyakov_best.csv to /home/submission/submission.csv


Score: 70.926150
Target: 68.896973
Gap: 2.029177


In [17]:
# Let's check the minimum distances in ALL preoptimized files
print('Checking minimum distances in all preoptimized files...')
print('='*70)

files = ['best_snapshot.csv', 'better_ensemble.csv', 'bucket_of_chump.csv', 
         'chistyakov_best.csv', 'ensemble_70_627.csv', 'saspav_best.csv', 'submission_70_926.csv']

for filename in files:
    try:
        df = pd.read_csv(f'/home/code/preoptimized/{filename}')
        parsed = parse_submission(df)
        
        min_dist = float('inf')
        min_group = None
        
        for n in range(1, 201):
            prefix = f'{n:03d}_'
            config_df = parsed[parsed['id'].str.startswith(prefix)]
            if len(config_df) != n:
                continue
            polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
            
            for i in range(len(polygons)):
                for j in range(i+1, len(polygons)):
                    dist = polygons[i].distance(polygons[j])
                    if dist < min_dist:
                        min_dist = dist
                        min_group = n
        
        print(f'{filename:25s}: min_dist = {min_dist:.2e} (group {min_group})')
    except Exception as e:
        print(f'{filename:25s}: ERROR - {e}')

Checking minimum distances in all preoptimized files...


best_snapshot.csv        : min_dist = 0.00e+00 (group 16)


better_ensemble.csv      : min_dist = 0.00e+00 (group 16)


bucket_of_chump.csv      : min_dist = 0.00e+00 (group 4)


chistyakov_best.csv      : min_dist = 1.09e-16 (group 2)


ensemble_70_627.csv      : min_dist = 0.00e+00 (group 2)


saspav_best.csv          : min_dist = 0.00e+00 (group 16)


submission_70_926.csv    : min_dist = 1.09e-16 (group 2)


In [18]:
# Let's try using bbox3 to optimize chistyakov_best.csv
# bbox3 might create better separation between trees

import subprocess
import os

# First, let's check bbox3 options
result = subprocess.run(['/home/code/bbox3', '--help'], capture_output=True, text=True)
print('bbox3 help:')
print(result.stdout)
print(result.stderr)

bbox3 help:
Loading submission.csv...
Initial score: 70.627589
Round 1/16
Score after round 1: 70.627589
Round 2/16
Score after round 2: 70.627589
Round 3/16
Score after round 3: 70.627589
Round 4/16
Score after round 4: 70.627589
Round 5/16
Score after round 5: 70.627589
Round 6/16
Score after round 6: 70.627589
Round 7/16
Score after round 7: 70.627589
Round 8/16
Score after round 8: 70.627589
Round 9/16
Score after round 9: 70.627589
Round 10/16
Score after round 10: 70.627589
Round 11/16
Score after round 11: 70.627589
Round 12/16
Score after round 12: 70.627589
Round 13/16
Score after round 13: 70.627589
Round 14/16
Score after round 14: 70.627589
Round 15/16
Score after round 15: 70.627589
Round 16/16
Score after round 16: 70.627589
Saving to submission.csv...
Final score: 70.627589




In [19]:
# Let's try a different approach: 
# Create a hybrid submission that uses chistyakov_best.csv for problematic groups
# and our better ensemble for groups that pass validation

# First, let's identify which groups in our current submission have very close trees
print('Identifying groups with very close trees in current submission...')

current_df = pd.read_csv('/home/code/preoptimized/best_snapshot.csv')
current_parsed = parse_submission(current_df)

close_groups = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = current_parsed[current_parsed['id'].str.startswith(prefix)]
    if len(config_df) != n:
        continue
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    
    min_dist = float('inf')
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            dist = polygons[i].distance(polygons[j])
            if dist < min_dist:
                min_dist = dist
    
    if min_dist < 1e-10:  # Very close
        close_groups.append((n, min_dist))

print(f'Groups with min_dist < 1e-10: {len(close_groups)}')
print('First 20:', close_groups[:20])

Identifying groups with very close trees in current submission...


Groups with min_dist < 1e-10: 181
First 20: [(2, 2.7967858445225423e-16), (3, 1.0986582014519777e-16), (4, 2.1350442781253004e-16), (5, 8.90259494096116e-15), (6, 7.580602652082601e-16), (7, 1.324697927109562e-16), (8, 1.2176424398683355e-16), (9, 8.155869142438646e-15), (10, 2.4213769636263838e-15), (11, 1.5986144032463192e-15), (12, 3.194543321111559e-14), (13, 2.7755575615628904e-16), (14, 8.210045890996083e-14), (16, 0.0), (17, 5.887846720064157e-17), (18, 1.373830901348303e-16), (19, 1.5700924586837752e-16), (20, 6.938893903907226e-17), (21, 0.0), (22, 1.4724925472180508e-16)]


In [20]:
# Check sample_submission.csv
print('Checking sample_submission.csv...')
sample_df = pd.read_csv('/home/code/preoptimized/sample_submission.csv')
sample_parsed = parse_submission(sample_df)

# Calculate score
total_score = 0.0
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = sample_parsed[sample_parsed['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    side = get_bounding_box_side(polygons)
    total_score += (side ** 2) / n

print(f'Score: {total_score:.6f}')

# Check minimum distances
min_dist = float('inf')
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = sample_parsed[sample_parsed['id'].str.startswith(prefix)]
    if len(config_df) != n:
        continue
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            dist = polygons[i].distance(polygons[j])
            if dist < min_dist:
                min_dist = dist

print(f'Minimum distance: {min_dist:.2e}')

Checking sample_submission.csv...


Score: 173.652299


Minimum distance: 4.09e-05


In [21]:
# Strategy: For each group, slightly scale down all trees toward the center
# This should create small gaps between trees without significantly affecting the score

def scale_config_toward_center(config_df, scale_factor=0.9999):
    """Scale all tree positions toward the center by a small factor."""
    # Calculate center of mass
    center_x = config_df['x'].mean()
    center_y = config_df['y'].mean()
    
    # Scale positions toward center
    new_df = config_df.copy()
    new_df['x'] = center_x + (config_df['x'] - center_x) * scale_factor
    new_df['y'] = center_y + (config_df['y'] - center_y) * scale_factor
    
    return new_df

# Test on group 008
print('Testing scale_config_toward_center on group 008...')
config_df = current_parsed[current_parsed['id'].str.startswith('008_')].copy()
print(f'Original:')
polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
print(f'  Min distance: {min(polygons[i].distance(polygons[j]) for i in range(len(polygons)) for j in range(i+1, len(polygons))):.2e}')
print(f'  Side length: {get_bounding_box_side(polygons):.6f}')

for scale in [0.9999, 0.999, 0.99, 0.98]:
    scaled_df = scale_config_toward_center(config_df, scale)
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in scaled_df.iterrows()]
    min_dist = min(polygons[i].distance(polygons[j]) for i in range(len(polygons)) for j in range(i+1, len(polygons)))
    side = get_bounding_box_side(polygons)
    
    # Check for overlaps
    has_overlap = False
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                has_overlap = True
                break
        if has_overlap:
            break
    
    print(f'Scale {scale}: min_dist={min_dist:.2e}, side={side:.6f}, overlaps={has_overlap}')

Testing scale_config_toward_center on group 008...
Original:
  Min distance: 1.22e-16
  Side length: 1.755921
Scale 0.9999: min_dist=0.00e+00, side=1.755871, overlaps=True
Scale 0.999: min_dist=0.00e+00, side=1.755422, overlaps=True
Scale 0.99: min_dist=0.00e+00, side=1.750930, overlaps=True
Scale 0.98: min_dist=0.00e+00, side=1.745938, overlaps=True


In [22]:
# Try scaling OUTWARD to create gaps
print('Testing scale OUTWARD on group 008...')
config_df = current_parsed[current_parsed['id'].str.startswith('008_')].copy()

for scale in [1.0001, 1.001, 1.01, 1.02]:
    scaled_df = scale_config_toward_center(config_df, scale)  # scale > 1 means outward
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in scaled_df.iterrows()]
    min_dist = min(polygons[i].distance(polygons[j]) for i in range(len(polygons)) for j in range(i+1, len(polygons)))
    side = get_bounding_box_side(polygons)
    
    # Check for overlaps
    has_overlap = False
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                has_overlap = True
                break
        if has_overlap:
            break
    
    print(f'Scale {scale}: min_dist={min_dist:.2e}, side={side:.6f}, overlaps={has_overlap}')

Testing scale OUTWARD on group 008...
Scale 1.0001: min_dist=7.50e-06, side=1.756054, overlaps=False
Scale 1.001: min_dist=7.50e-05, side=1.757250, overlaps=False
Scale 1.01: min_dist=7.50e-04, side=1.769214, overlaps=False
Scale 1.02: min_dist=1.50e-03, side=1.782508, overlaps=False


In [23]:
# Apply scale outward to all groups and calculate new score
print('Applying scale outward (1.0001) to all groups...')

scale_factor = 1.0001
new_data = []
new_score = 0.0

for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = current_parsed[current_parsed['id'].str.startswith(prefix)].copy()
    
    if len(config_df) != n:
        print(f'WARNING: Group {n} has {len(config_df)} trees')
        continue
    
    # Scale outward
    center_x = config_df['x'].mean()
    center_y = config_df['y'].mean()
    config_df['x'] = center_x + (config_df['x'] - center_x) * scale_factor
    config_df['y'] = center_y + (config_df['y'] - center_y) * scale_factor
    
    # Calculate new side length
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    side = get_bounding_box_side(polygons)
    new_score += (side ** 2) / n
    
    # Add to new data
    for _, row in config_df.iterrows():
        new_data.append({
            'id': row['id'],
            'x': row['x'],
            'y': row['y'],
            'deg': row['deg']
        })

print(f'Original score: 70.627582')
print(f'New score with scale 1.0001: {new_score:.6f}')
print(f'Score increase: {new_score - 70.627582:.6f}')

Applying scale outward (1.0001) to all groups...


Original score: 70.627582
New score with scale 1.0001: 70.640098
Score increase: 0.012516


In [24]:
# Create the scaled submission and verify no overlaps
print('Creating scaled submission...')

scale_factor = 1.0001
new_data = []

for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = current_parsed[current_parsed['id'].str.startswith(prefix)].copy()
    
    if len(config_df) != n:
        continue
    
    # Scale outward
    center_x = config_df['x'].mean()
    center_y = config_df['y'].mean()
    config_df['x'] = center_x + (config_df['x'] - center_x) * scale_factor
    config_df['y'] = center_y + (config_df['y'] - center_y) * scale_factor
    
    for _, row in config_df.iterrows():
        new_data.append({
            'id': row['id'],
            'x': row['x'],
            'y': row['y'],
            'deg': row['deg']
        })

scaled_df = pd.DataFrame(new_data)

# Verify no overlaps
print('Verifying no overlaps...')
overlap_groups = []
min_distances = []

for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = scaled_df[scaled_df['id'].str.startswith(prefix)]
    if len(config_df) != n:
        continue
    
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    
    # Check overlaps
    has_overlap = False
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                has_overlap = True
                break
        if has_overlap:
            break
    
    if has_overlap:
        overlap_groups.append(n)
    
    # Find min distance
    if n > 1:
        min_dist = min(polygons[i].distance(polygons[j]) for i in range(len(polygons)) for j in range(i+1, len(polygons)))
        min_distances.append((n, min_dist))

print(f'Groups with overlaps: {len(overlap_groups)}')
if overlap_groups:
    print(f'Overlap groups: {overlap_groups[:20]}')

min_distances.sort(key=lambda x: x[1])
print(f'Smallest distances: {min_distances[:10]}')

Creating scaled submission...


Verifying no overlaps...


Groups with overlaps: 91
Overlap groups: [5, 10, 11, 13, 17, 18, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
Smallest distances: [(5, 0.0), (10, 0.0), (11, 0.0), (13, 0.0), (17, 0.0), (18, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0)]


In [25]:
# Check which groups have actual overlaps in best_snapshot.csv
print('Checking for actual overlaps in best_snapshot.csv...')

overlap_groups = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = current_parsed[current_parsed['id'].str.startswith(prefix)]
    if len(config_df) != n:
        continue
    
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                overlap_groups.append(n)
                break
        else:
            continue
        break

print(f'Groups with actual overlaps: {len(overlap_groups)}')
print(f'Groups: {overlap_groups}')

Checking for actual overlaps in best_snapshot.csv...


Groups with actual overlaps: 8
Groups: [16, 61, 70, 79, 98, 99, 102, 138]


In [26]:
# Create hybrid submission: best_snapshot.csv + chistyakov_best.csv for overlap groups
print('Creating hybrid submission...')

# Load both submissions
best_df = pd.read_csv('/home/code/preoptimized/best_snapshot.csv')
best_parsed = parse_submission(best_df)

chistyakov_df = pd.read_csv('/home/code/preoptimized/chistyakov_best.csv')
chistyakov_parsed = parse_submission(chistyakov_df)

# Groups with overlaps in best_snapshot.csv
overlap_groups = [16, 61, 70, 79, 98, 99, 102, 138]

# Create hybrid
hybrid_data = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    
    if n in overlap_groups:
        # Use chistyakov for overlap groups
        config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith(prefix)]
        source = 'chistyakov'
    else:
        # Use best_snapshot for non-overlap groups
        config_df = best_parsed[best_parsed['id'].str.startswith(prefix)]
        source = 'best_snapshot'
    
    for _, row in config_df.iterrows():
        hybrid_data.append({
            'id': row['id'],
            'x': row['x'],
            'y': row['y'],
            'deg': row['deg']
        })

hybrid_df = pd.DataFrame(hybrid_data)

# Calculate score
total_score = 0.0
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = hybrid_df[hybrid_df['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    side = get_bounding_box_side(polygons)
    total_score += (side ** 2) / n

print(f'Hybrid score: {total_score:.6f}')
print(f'best_snapshot score: 70.627582')
print(f'chistyakov score: 70.926150')

# Verify no overlaps
overlap_count = 0
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = hybrid_df[hybrid_df['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                overlap_count += 1
                break
        else:
            continue
        break

print(f'Groups with overlaps: {overlap_count}')

Creating hybrid submission...


Hybrid score: 70.637494
best_snapshot score: 70.627582
chistyakov score: 70.926150


Groups with overlaps: 0


In [ ]:
# Check minimum distances in hybrid submission
print('Checking minimum distances in hybrid submission...')

min_distances = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = hybrid_df[hybrid_df['id'].str.startswith(prefix)]
    if len(config_df) < 2:
        continue
    
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    
    min_dist = float('inf')
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            dist = polygons[i].distance(polygons[j])
            if dist < min_dist:
                min_dist = dist
    
    min_distances.append((n, min_dist))

min_distances.sort(key=lambda x: x[1])
print('Smallest distances:')
for n, dist in min_distances[:30]:
    source = 'chistyakov' if n in overlap_groups else 'best_snapshot'
    print(f'  Group {n:03d} ({source}): {dist:.2e}')